In [2]:
import requests
import pandas as pd
import numpy as np

In [5]:
url = "https://api-football-v1.p.rapidapi.com/v3/timezone"

headers = {
"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
"X-RapidAPI-Key": "SIGN-UP-FOR-KEY"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"message":"You are not subscribed to this API."}
